# Step 1
L'idea è quella di utilizzare l'informazione nota della massa della $Z^0$ per poter riuscire ad aumentare l'efficienza di muoni selezionati, in particolare il nostro obiettivo è quello di abbattere il fondo e il noise. Di seguito mostro quell che dovrebbe essere l'idea generale, sperando di non fotttere tutto
* Grafico la massa ricostruita utilizzando due $\mu$ che siano OS, qualsiasi segnale chiamo in questa descrizione come rumore intendo gli SS in tutto il range
* Ricerco vari tagli da poter effettuare così da selezionare solo determinati muoni
    + Utilizzo dunque un metodo del tipo _tag and probe_ 
        - Studiando i file DYJets e TTDilep Cerco di capire i migliori range dove effettuare dei tagli
        - Seleziono per ogni evento i muoni "tag" e poi con criteri meno stringenti cerco un probe
        - Il solo utilizzare questo metodo dovrebbe garantire un certo abbattimento del noise, dato che non sto pigliando tutti i muoni a caso senza senso
* Rifaccio il grafico della massa, studiandomi in particolare le OS (che già so essere un segnale)  dopo che ho effettuato tutti i tagli che ho nuovamente ritrovato, sperando che ciò vadad bene

In [1]:
import ROOT
import particle as pt

Welcome to JupyROOT 6.26/04


In [2]:
fileDY = ROOT.TFile("./datas/DYJets.root","OPEN")
dytr = fileDY.trees.events

fileTT = ROOT.TFile("./datas/TTDilep.root","OPEN")
tttr = fileTT.trees.events

file = ROOT.TFile("./datas/Data.root","OPEN")
tr = file.trees.events

## DYJets

In [ ]:
are_os = lambda x,y: x.charge*y.charge<0
h_mu_mass_os = ROOT.TH1F("DY_mu_mass","DY Muon mass",150,0,150) 
h_mu_mass_ss = ROOT.TH1F("DY_mu_mass","DY Muon mass",150,0,150)

c=ROOT.TCanvas()

for i in range(dytr.GetEntries()):
    dytr.GetEntry(i)
    muons=pt.get_collection(dytr,"muontracks")
    for mi in range(len(muons)):
        for mj in range(mi):
            if(are_os(muons[mi],muons[mj])):
                h_mu_mass_os.Fill((muons[mi].p4+muons[mj].p4).M())
            else:
                h_mu_mass_ss.Fill((muons[mi].p4+muons[mj].p4).M())
c.Draw()
c.SetLogy()
h_mu_mass_os.Draw()
h_mu_mass_ss.SetLineColor(ROOT.kRed)
h_mu_mass_ss.Draw("same")

### Histograms for find cutting

In [ ]:
pt.mass_z_range(fileDY, dytr, "muontracks_chi2", 100, 0, 2,1)

In [ ]:
pt.mass_z_range(fileDY, dytr, "muontracks_dz", 100, 0, 0.01,1)

In [ ]:
pt.mass_z_range(fileDY, dytr, "muontracks_dB", 100, 0, 0.006,1)

In [ ]:
pt.mass_z_range(fileDY, dytr, "muontracks_isoDeposits", 100, 0.1, 10,1)

---
### Cut
Procedo col cutting dove farò i seguenti tagli:

| Caratteristica | Criterio stretto | Criterio largo |
| ------ | ----------- ||
| $\chi^2$   | $0.6<\chi^2<1.2$ | $0.2<\chi^2<1.8$ |
| $dz$ | $<0.003$ | $<0.005$ |
| $dB$ | $<0.0015$ | $<0.003$|
| $iso$ | $<2$ | $<4$ | 

Dopo questo primo tentativo calcolo l'efficienza e qunindi provo a fare uno scan dei tagli, dunque seguo i seguenti passaggi
+ Tentativo di cut a caso dul DYJ
+ Scan dei vari tagli possibili e quindi provare varie efficienze
    - Forse sarà necessario creare un istogramma di efficienze



In [ ]:
h_passing_probes = ROOT.TH1F("DY_mu_mass_PP","DY Muon mass PP",150,0,150) 
h_failing_probes = ROOT.TH1F("DY_mu_mass_PP","DY Muon mass PP",150,0,150) 
c5=ROOT.TCanvas()

n_pass = 0
n_fail = 0

for i in range(dytr.GetEntries()):
    dytr.GetEntry(i)
    muons = pt.get_collection(dytr, "muontracks")
    for mi in range(len(muons)):
        #exists_tag=0
        if(dytr.muontracks_chi2[mi]<1.2 and dytr.muontracks_chi2[mi]>0.6 and dytr.muontracks_dz[mi]<0.003 and dytr.muontracks_dB[mi]<0.0015 and dytr.muontracks_isoDeposits[mi]<2):
            p4t=muons[mi].p4
            #exists_tag=1     
            for mj in range(len(muons)): #devo mettere che arrivi ad essere uguale a mi e non minore
                if(mj != mi):
                    if(dytr.muontracks_chi2[mj]<1.8 and dytr.muontracks_chi2[mj]>0.2 and dytr.muontracks_dz[mj]<0.005 and dytr.muontracks_dB[mj]<0.003 and dytr.muontracks_isoDeposits[mj]<4):
                        p4pp=muons[mj].p4
                        s_pass = (p4t+p4pp).M()
                        h_passing_probes.Fill(s_pass)
                        n_pass = n_pass +1
                    else:
                        p4fp=muons[mj].p4
                        s_fail = (p4t+p4fp).M()
                        h_failing_probes.Fill(s_fail)
                        n_fail = n_fail +1
c5.Draw()
h_passing_probes.Draw()
h_failing_probes.SetLineColor(ROOT.kRed)
h_failing_probes.Draw("SAME")

epsilon = eff(n_pass, n_fail)
print((n_pass, n_fail))
print(epsilon)

In [ ]:
n_pass = 0
n_fail = 0

for i in range(dytr.GetEntries()):
    dytr.GetEntry(i)
    muons = pt.get_collection(dytr, "muontracks")
    for mi in range(len(muons)):
        if(dytr.muontracks_chi2[mi]<1.2 and dytr.muontracks_chi2[mi]>0.6 and dytr.muontracks_dz[mi]<0.003 and dytr.muontracks_dB[mi]<0.0015 and dytr.muontracks_isoDeposits[mi]<2):
            p4t=muons[mi].p4
            